In [47]:
import numpy as np
from numpy import nan
from tabulate import tabulate


### Environnement :
- Initialisation de 2 stack
- Affichage de 2 Stack 
    - (print du top aux bot -> Ok)
    - Afficher toutes les valeurs si l'autre ne possede pas la meme taille -> KO
    - afficher les valeurs NULL -> KO (le "-" est pourris)
- push A fonctionnent mais trouver une facons de renplir la stack qui pet un element



In [169]:
class Env():
    
    def __init__(self, size):
        self.stack_A = np.arange(size)
        self.stack_B = np.full(size, -99999)
        
    def print_stack(self):
        stack = list()
        for i in reversed(range(self.stack_A.size)):
            number_A = self.stack_A[i]
            if not self.stack_A.size or self.stack_A[i] == -99999:
                number_A = "-"
            else:
                number_A = self.stack_A[i]
            if not self.stack_B.size or self.stack_B[i] == -99999:
                number_B = "-"
            else:
                number_B = self.stack_B[i]
            stack.append([number_A, number_B])
        
        #print(table)
        print(tabulate(stack, headers=["A", "B"]))
    
    def push_a(self):
        self.stack_B[-1] = self.stack_A[-1]
        self.stack_A = np.delete(self.stack_A, self.stack_A[-1])
        self.stack_A = np.append(self.stack_A, -99999)
        self.print_stack()

In [170]:
test = Env(10)

In [171]:
test.print_stack()
test.push_a()

  A  B
---  ---
  9  -
  8  -
  7  -
  6  -
  5  -
  4  -
  3  -
  2  -
  1  -
  0  -
A    B
---  ---
-    9
8    -
7    -
6    -
5    -
4    -
3    -
2    -
1    -
0    -
